In [1]:
import datetime
from google.cloud import bigquery

client = bigquery.Client()

dataset_name = 'quipu_dbt_zeph_dev'
project_name = 'zs-econ-data'

### Permits Timeseries

In [61]:
rolling_365_permits = client.query(f'''
    SELECT 
        date
        , AVG(total_sqft_issued_amt) OVER (ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND 364 FOLLOWING) AS sqft_per_permit
        , AVG(total_new_units_issued_amt) OVER (ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND 364 FOLLOWING) AS total_new_units_issued_amt
        , AVG(count_permits_issued) OVER (ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND 364 FOLLOWING) AS count_permits_issued
        , SUM(count_permits_issued) OVER (ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND 364 FOLLOWING) AS count_total_permits_issued
    FROM `{project_name}.{dataset_name}.permits_by_date`
    WHERE date > '2010-01-01'
    ORDER BY date ASC
''').result().to_dataframe()

In [57]:
rolling_365_permits[100:].set_index('date').to_csv('../data/issuances_rolling_365.csv')

### Yesterday Permits Table

In [37]:
yesterday_permits_issued = client.query(f'''
    SELECT
        structure_type
        , work_type
        , function_type
        , neighborhood
        , total_sqft_amt
        , stories_amt
        , new_units_amt
        , value
        , description
    FROM `{project_name}.{dataset_name}.permits_cleaned`
    WHERE DATE(issued_at) BETWEEN DATE(CURRENT_DATE - 4) AND DATE(CURRENT_DATE)
''').result().to_dataframe()\
.rename(columns={
    'structure_type':'Structure'
    ,'work_type':'Type of Work'
    ,'function_type':'Building Function'
    ,'neighborhood':'Neighborhood'
    ,'total_sqft_amt':'New Square Feet'
    ,'stories_amt':'Building\'s Stories'
    ,'new_units_amt':'New Residential Units'
    ,'value':'Construction Value'
    ,'description':'Description'})
yesterday_permits_issued['Neighborhood'] = yesterday_permits_issued['Neighborhood']\
                                            .str.replace(' Neighborhood','').str.replace(' Association','')

In [32]:
yesterday_permits_issued.sort_values('New Square Feet', ascending=False).to_csv(f'../data/table_1_issued.csv', index=False)

### Avg Days to Issue

In [209]:
# days_to_issue = client.query(f'''
#     SELECT 
#         date
#         , days_to_issue_sqft
#         , days_to_issue_new_units
#         , avg_days_to_issue
#     FROM `{project_name}.{dataset_name}.permits_by_date`
#     WHERE date > '2010-01-01'
#     ORDER BY date ASC
# ''').result().to_dataframe()

In [53]:
# dfs = {}
# for field in ['days_to_issue_sqft', 'days_to_issue_new_units', 'avg_days_to_issue']:
#     dfs[field] = days_to_issue[['date',field]]\
#                     .set_index('date').ffill().rolling(365).mean()[365:]\
#                     .rename(columns={field:'value'})
#     dfs[field].to_csv(f'data/graph_2_{field}.csv')
#     dfs[field].plot(figsize=(12,3))

### Permits Backlog

In [54]:
# backlog = client.query(f'''
#     SELECT 
#         date
#         , AVG(total_sqft_waiting_approval) OVER (ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND 364 FOLLOWING) AS sqft_backlog_rolling_year
#         , AVG(total_sqft_issued_amt) OVER (ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND 364 FOLLOWING) AS sqft_issued_rolling_year
#         , AVG(total_sqft_review_amt) OVER (ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND 364 FOLLOWING) AS sqft_submitted_rolling_year
#         , AVG(total_new_units_waiting_approval) OVER (ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND 364 FOLLOWING) AS new_units_backlog_rolling_year
#         , AVG(total_new_units_issued_amt) OVER (ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND 364 FOLLOWING) AS new_units_issued_rolling_year
#         , AVG(total_new_units_review_amt) OVER (ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND 364 FOLLOWING) AS new_units_submitted_rolling_year
#     FROM `{project_name}.{dataset_name}.permits_by_date`
#     WHERE date > '2010-01-01'
#     ORDER BY date ASC
# ''').result().to_dataframe()
# backlog['sqft_backlog_rolling_year_pct'] = backlog.sqft_backlog_rolling_year/backlog.sqft_issued_rolling_year
# backlog['units_backlog_rolling_year_pct'] = backlog.sqft_backlog_rolling_year/backlog.sqft_issued_rolling_year

In [51]:
# issues_and_submissions_sqft = backlog[['date','sqft_issued_rolling_year','sqft_submitted_rolling_year']][365:]\
#     .set_index('date')
# issues_and_submissions_sqft.plot(figsize=(12,3))
# # issues_and_submissions_sqft.to_csv('data/graph_2_issues_and_submissions_sqft.csv')

# sqft_backlog_num = backlog[['date','sqft_backlog_rolling_year']][365:].set_index('date')
# sqft_backlog_num.plot(figsize=(12,3))
# # sqft_backlog_num.to_csv('data/graph_2_sqft_backlog_num.csv')

# sqft_backlog_pct = backlog[['date','sqft_backlog_rolling_year_pct']][365:].set_index('date')
# sqft_backlog_pct.plot(figsize=(12,3))
# # sqft_backlog_pct.to_csv('data/graph_2_sqft_backlog_pct')

In [52]:
# issues_and_submissions_units = backlog[['date','new_units_issued_rolling_year','new_units_submitted_rolling_year']][365:]\
#     .set_index('date')
# issues_and_submissions_units.plot(figsize=(12,3))
# # issues_and_submissions_units.to_csv('data/graph_2_issues_and_submissions_units.csv')

# units_backlog_num = backlog[['date','new_units_backlog_rolling_year']][365:].set_index('date')
# units_backlog_num.plot(figsize=(12,3))
# # units_backlog_num.to_csv('data/graph_2_units_backlog_num.csv')

# units_backlog_pct = backlog[['date','units_backlog_rolling_year_pct']][365:].set_index('date')
# units_backlog_pct.plot(figsize=(12,3))
# # units_backlog_pct.to_csv('data/graph_2_units_backlog_pct')

### Issuances By [Type]

In [7]:
dfs = {}
for permit_type in ['work_type','structure_type','function_type']:
    dfs[permit_type] = client.query(f'''
        SELECT 
            date
            , {permit_type} AS type
            , total_sqft_issued_amt AS value
        FROM `{project_name}.{dataset_name}.permits_by_date_and_{permit_type}`
        WHERE date > '2010-01-01'
            AND {permit_type} IS NOT NULL
        ORDER BY date ASC
    ''').result().to_dataframe()
    dfs[permit_type].pivot(
        index='date'
        ,columns='type'
        ,values='value'
    ).fillna(0).rolling(365).mean().reset_index().melt(
                                                    id_vars=['date']
                                                    , var_name='type'
                                                    , value_name='value')\
    .to_csv(f'../data/issuances_x_{permit_type}.csv')

### HTML Construction

In [35]:
yesterday_permits = client.query(f'''
    SELECT 
        *
    FROM `{project_name}.{dataset_name}.permits_cleaned`
    WHERE DATE(issued_at, "America/Los_Angeles") = (CURRENT_DATE - 4)
        AND total_sqft_amt >= 200
''').result().to_dataframe()

In [77]:
permits_by_date = client.query(f'''
    SELECT 
        *
        , EXTRACT(DAYOFWEEK FROM date) AS day_of_week
        , EXTRACT(DAYOFWEEK FROM date) = EXTRACT(DAYOFWEEK FROM (CURRENT_DATE - 4)) AS is_same_day_of_week_as_yesterday
        , (DATE(date) = (CURRENT_DATE - 4)) AS is_yesterday
    FROM `{project_name}.{dataset_name}.permits_by_date`
    WHERE date BETWEEN '2022-01-01' AND CURRENT_DATE
''').result().to_dataframe()

In [78]:
all_days_permits_sqft = permits_by_date[permits_by_date.is_yesterday==False]['total_sqft_issued_amt'].mean()
yesterday_permits_sqft = permits_by_date[permits_by_date.is_yesterday]['total_sqft_issued_amt'].values[0]
same_day_as_yesterday_permits_sqft = permits_by_date[
    (permits_by_date.is_same_day_of_week_as_yesterday==True) & (permits_by_date.is_yesterday==False)
]['total_sqft_issued_amt'].mean()
yesterday_permits_sqft_to_average = same_day_as_yesterday_permits_sqft/yesterday_permits_sqft
yesterday_permits_w_swft = permits_by_date[permits_by_date.is_yesterday].count_permits_issued_w_sqft.values[0]

In [85]:
new_html = f'''<!DOCTYPE HTML>
<html>
<head>
    <!-- Sunday, Feb 27, 2022 08:37 -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title id="title">Projects</title>
    <link rel="stylesheet" href="/styles/styles_projects.css" type="" media="screen" />
  <script src="http://d3js.org/d3.v3.min.js" charset="utf-8"></script>
    <script src="https://d3js.org/d3.v4.js"></script>
</head>
<body>
    <h1 style='margin-top:50px;text-align:center;margin-left:20px;'>Portland Development Report</h1>
    <p style='text-align:center;font-size:12px;'>
        Last Updated: {datetime.datetime.now().strftime('%A, %b %d, %Y')}
        <br>
        Source: City of Portland, <a href='https://www.portlandmaps.com/development/'>Portland Maps API</a>
        <br>
        Email zephyr.schafer@gmail.com with data requests or inquiries.
    </p>


    <p style='margin-left:10%;margin-right:10%;text-align:left;font-size:16px;'>
        To add to the City's developed supply of space, new construction projects require approval
        from the City's <a href='https://www.portland.gov/bds'>Bureau of Development Services</a>.
        This daily report tracks the pace of development via the volume of permits.
        <br><br>
    <p style='margin-left:10%;margin-right:10%;text-align:left;font-size:16px;'>
        For the 365-day period ending on {datetime.datetime.now().strftime('%A, %b %d, %Y')}, 
        the City issued a total of {'{:,.0f}'.format(rolling_365_permits[-1:].count_total_permits_issued.values[0])} permits 
        with at least 200 new square feet of development
        , or {'{:,.1f}'.format(rolling_365_permits[-1:].count_permits_issued.values[0])} per day.
         On average, these allowed for {'{:,.0f}'.format(rolling_365_permits[-1:].sqft_per_permit.values[0])} 
        new square feet of development, and {'{:,.0f}'.format(rolling_365_permits[-1:].total_new_units_issued_amt.values[0])} new residential units.
        The graph below shows permit issuances from 2010 through today.
    </p>
    <div class='headerWithTooltip'>
        <h4 class='headerWithTooltip' style='text-align:center;'>Average Daily Permits Issued (rolling 365-day mean)
            <span class="tooltiptext">The graph shows a rolling 365-day average. Only permits with at least 200 square feet of affected construction are included.</span>
        </h4>
    </div>
    <div id="graph"></div>
    <div id="selector"></div>

    <p style='margin-left:10%;margin-right:10%;text-align:left;font-size:16px;'>
        Yesterday, ({datetime.datetime.now().strftime('%A, %b %d, %Y')}) the City of Portland 
        issued permits for {'{:,.1f}'.format(yesterday_permits_sqft/(10**3))} K new square footage of development.
        (That's {'{:,.1f}'.format(yesterday_permits_sqft_to_average*100)}% of the 
        average for a {datetime.datetime.now().strftime('%A')}
        for the past 365 days.)
        <br><br>
        The table below describes the permits issued yesterday
    </p>

    <div style='margin-left:10%;margin-right:10%;text-align:left;font-size:14px;' id="yesterday_permits"></div>

    <p style='margin-left:10%;margin-bottom:50px;margin-top:75px;margin-right:10%;text-align:left;font-size:16px;'>
        Building permits vary by the type of structure, its function, and the type
        of construction. The stacked area chart below shows a rolling 365-day average
        of permits by type and date if issue.
    </p>

    <div style='margin-left:10%;margin-right:10%;text-align:left;font-size:14px;' id="permits-stacked-by-type-viewof-selector"></div>
    <div style='margin-left:10%;margin-right:10%;text-align:left;font-size:14px;' id="permits-stacked-by-type-key"></div>
    <div style='margin-left:10%;margin-right:10%;text-align:left;font-size:14px;' id="permits-stacked-by-type-chart"></div>
    <div style='margin-top:20%'></div>


    <script type='module' src='/scripts/permits_report/yesterday_permits_table.js'></script>
    <script type='module' src='/scripts/permits_report/permits_timeseries.js'></script>
    <script type='module' src='/scripts/permits_report/permits_stacked_by_type.js'></script>

<html>'''

In [84]:
file_write = open("portland_permits.html", "wt")
n = file_write.write(new_html)
file_write.close()